<a href="https://colab.research.google.com/github/nannthd/project_AIEngineer/blob/main/license_plate_EasyOCR_edit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Install YOLOv8

In [2]:
# Pip install method (recommended)

!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.2/107.7 GB disk)


In [ ]:
!pip install pytube opencv-python-headless

In [4]:
from ultralytics import YOLO
from IPython.display import display, Image

#Data

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="AWtZRbYK04sFZNNZ1t4X")
project = rf.workspace("projectaiengineer").project("license-plates-gpj5f")
version = project.version(2)
dataset = version.download("yolov8")

In [ ]:
%cat {dataset.location}/data.yaml

In [7]:
!sed -i '$d' {dataset.location}/data.yaml
!sed -i '$d' {dataset.location}/data.yaml
!sed -i '$d' {dataset.location}/data.yaml
!sed -i '$d' {dataset.location}/data.yaml
!echo -e "test: ../test/images\ntrain: ../train/images\nval: ../valid/images" >> {dataset.location}/data.yaml

In [ ]:
%cat {dataset.location}/data.yaml

# Load a model

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Load a model
model = YOLO("/content/drive/MyDrive/license plate/best.pt")

In [11]:
model.names

{0: 'number', 1: 'province'}

In [ ]:
image_test = '/content/-license-plates-2/test/images'

In [ ]:
pred = model.predict(image_test, save=True, save_txt=True)
pred

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

directory = '/content/runs/detect/predict'

# ตรวจสอบว่าไดเรกทอรีนี้มีไฟล์ภาพหรือไม่
if os.path.isdir(directory):
    # หากมีไฟล์ภาพในไดเรกทอรี
    image_files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

    # แสดงภาพทีละภาพ
    for image_file in image_files:
        image_path = os.path.join(directory, image_file)
        img = mpimg.imread(image_path)

        plt.imshow(img)
        plt.title(image_file)
        plt.axis('off')  # ปิดแกน X และ Y
        plt.show()
else:
    print("ไม่พบไดเรกทอรีที่ระบุ")

#EasyOCR

## อ่านไฟล์ th.txt

In [ ]:
!pip install ultralytics

In [ ]:
!pip install easyocr

In [ ]:
!pip install opencv-python-headless

In [ ]:
!git clone https://github.com/nannthd/EasyOCR_license_plate

In [13]:
%cd /content/EasyOCR_license_plate

/content/EasyOCR_license_plate


In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install Levenshtein

In [ ]:
# Step 1: Install YOLOv8, EasyOCR, and OpenCV
# !pip install ultralytics
# !pip install easyocr
# !pip install opencv-python-headless

# Step 2: Import Required Libraries
import cv2
import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt
import easyocr

# Step 3: Load the YOLOv8 Model with Custom Weights
model = YOLO('/content/drive/MyDrive/license plate/best.pt')  # Replace with the path to your weights file

# Step 4: Load the Input Image
input_image_path = '/content/-license-plates-2/valid/images/CH1_20150510191746_-1436_jpg.rf.5a0ed401cdbaf5df27a97f86ef0ed7df.jpg'  # Replace with the path to your input image
image = cv2.imread(input_image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

# Step 5: Perform Inference
results = model.predict(image_rgb, save=True)

# Step 6: Initialize EasyOCR Reader for Thai Language
reader = easyocr.Reader(['th'])  # Specify 'th' for Thai language

# Step 7: Process Each Detection and Apply EasyOCR
output_image = image_rgb.copy()
detected_texts = {'text_number': [], 'text_province': []}  # Dictionary to store texts by category
cropped_images = []

for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Access the bounding box coordinates
        class_id = int(box.cls)  # Access the class ID

        # Draw bounding box
        cv2.rectangle(output_image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Extract the detected region and convert to grayscale
        detected_region = cv2.cvtColor(image_rgb[y1:y2, x1:x2], cv2.COLOR_RGB2GRAY)
        cropped_images.append((class_id, detected_region))

        # Apply EasyOCR to the grayscale region
        ocr_result = reader.readtext(detected_region)

        # Collect OCR result and categorize by class
        for (bbox, text, prob) in ocr_result:
            if class_id == 0:
                detected_texts['text_number'].append(text)
            elif class_id == 1:
                detected_texts['text_province'].append(text)

# Step 8: Display the Output Image with Bounding Boxes
plt.figure(figsize=(10, 10))
plt.imshow(output_image)
plt.axis('off')
plt.show()

# Step 9: Display Cropped Images of Each Class
for idx, (class_id, cropped_image) in enumerate(cropped_images):
    plt.figure(figsize=(5, 5))
    plt.title(f'Class ID: {class_id}')
    plt.imshow(cropped_image, cmap='gray')
    plt.axis('off')
    plt.show()

# Step 10: Print the OCR Results
print("Detected Texts - Number:")
for text in detected_texts['text_number']:
    print(text)

print("\nDetected Texts - Province:")
for text in detected_texts['text_province']:
    print(text)

In [ ]:
%cd /content/EasyOCR_license_plate
# Step 1: Install YOLOv8, EasyOCR, and OpenCV if not installed

# Step 2: Import Required Libraries
import sys
import cv2
import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt
import Levenshtein as lev  # นำเข้าไลบรารี Levenshtein
from Levenshtein import distance

# เพิ่มเส้นทางไปยัง EasyOCR_license_plate
sys.path.append('/content/EasyOCR_license_plate')

# นำเข้า Reader จาก EasyOCR_license_plate
from easyocr import Reader

# Step 3: Load the YOLOv8 Model with Custom Weights
model = YOLO('/content/drive/MyDrive/license plate/best.pt')  # เปลี่ยนเป็นพาธของไฟล์ weights ของคุณ

# Step 4: Load the Input Image
input_image_path = '/content/-license-plates-2/valid/images/CH1_20150510191746_-1436_jpg.rf.5a0ed401cdbaf5df27a97f86ef0ed7df.jpg'  # เปลี่ยนเป็นพาธของไฟล์ภาพของคุณ
image = cv2.imread(input_image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # แปลงจาก BGR เป็น RGB

# Step 5: Perform Inference
results = model.predict(image_rgb, save=True)

# Step 6: Initialize EasyOCR Reader for Thai Language and specify dictionary path
# พาธไฟล์ th.txt
th_dict_path = '/content/EasyOCR_license_plate/easyocr/dict/th.txt'

# เปิดไฟล์ th.txt เพื่อดึงข้อมูลคำศัพท์
with open(th_dict_path, 'r', encoding='utf-8') as f:
    thai_words = [word.strip() for word in f.readlines()]

# Step 7: Process Each Detection and Apply EasyOCR
output_image = image_rgb.copy()
detected_texts = {'text_number': [], 'text_province': []}
cropped_images = []

for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        class_id = int(box.cls)

        cv2.rectangle(output_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        detected_region = cv2.cvtColor(image_rgb[y1:y2, x1:x2], cv2.COLOR_RGB2GRAY)
        cropped_images.append((class_id, detected_region))

        ocr_result = reader.readtext(detected_region, detail=0)

        selected_texts = []
        for text in ocr_result:
            closest_word = text

            # หาคำใกล้เคียงจากไฟล์ th.txt โดยใช้ระยะทาง Levenshtein
            if class_id == 1:  # เช็คเฉพาะ Province เท่านั้น
                min_distance = float('inf')
                best_match = text

                for th_word in thai_words:
                    dist = lev.distance(text, th_word)
                    if dist < min_distance:
                        min_distance = dist
                        best_match = th_word

                selected_texts.append(best_match)
            else:
                selected_texts.append(text)

        if class_id == 0:
            detected_texts['text_number'].extend(selected_texts)
        elif class_id == 1:
            detected_texts['text_province'].extend(selected_texts)

# แสดงภาพผลลัพธ์พร้อมกล่อง bounding box
plt.figure(figsize=(10, 10))
plt.imshow(output_image)
plt.axis('off')
plt.show()

# แสดงภาพที่ตัดออกมาแต่ละประเภท
for idx, (class_id, cropped_image) in enumerate(cropped_images):
    plt.figure(figsize=(5, 5))
    plt.title(f'Class ID: {class_id}')
    plt.imshow(cropped_image, cmap='gray')
    plt.axis('off')
    plt.show()


# แสดงผลลัพธ์ที่ตรวจจับได้และคำที่ถูกแทนที่จากไฟล์ th.txt เฉพาะ Province
print("Detected Texts - Number:")
for text in detected_texts['text_number']:
    print(text)

print("\nDetected Texts - Province:")
for text in detected_texts['text_province']:
    print(text)

###ปรับภาพ

In [ ]:
%cd /content/EasyOCR_license_plate
# Step 1: Install YOLOv8, EasyOCR, and OpenCV if not installed

# Step 2: Import Required Libraries
import sys
import cv2
import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt
import Levenshtein as lev  # นำเข้าไลบรารี Levenshtein
from Levenshtein import distance

# เพิ่มเส้นทางไปยัง EasyOCR_license_plate
sys.path.append('/content/EasyOCR_license_plate')

# นำเข้า Reader จาก EasyOCR_license_plate
from easyocr import Reader

# Step 3: Load the YOLOv8 Model with Custom Weights
model = YOLO('/content/drive/MyDrive/license plate/best.pt')  # เปลี่ยนเป็นพาธของไฟล์ weights ของคุณ

# Step 4: Load the Input Image
input_image_path = '/content/-license-plates-2/valid/images/CH1_20150510131335_-5922_jpg.rf.76da97dcd3c09e9fcc5c4c3bc44903a5.jpg'  # เปลี่ยนเป็นพาธของไฟล์ภาพของคุณ
image = cv2.imread(input_image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # แปลงจาก BGR เป็น RGB

# Adjust contrast
alpha = 2  # Contrast control (1.0-3.0)
adjusted_image = cv2.convertScaleAbs(image_rgb, alpha=alpha, beta=0)

# Step 5: Perform Inference
results = model.predict(adjusted_image)

# Step 6: Initialize EasyOCR Reader for Thai Language and specify dictionary path
# พาธไฟล์ th.txt
th_dict_path = '/content/EasyOCR_license_plate/easyocr/dict/th.txt'

# เปิดไฟล์ th.txt เพื่อดึงข้อมูลคำศัพท์
with open(th_dict_path, 'r', encoding='utf-8') as f:
    thai_words = [word.strip() for word in f.readlines()]

# Step 7: Process Each Detection and Apply EasyOCR
output_image = adjusted_image.copy()
detected_texts = {'text_number': [], 'text_province': []}
cropped_images = []

for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        class_id = int(box.cls)

        cv2.rectangle(output_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        detected_region = cv2.cvtColor(adjusted_image[y1:y2, x1:x2], cv2.COLOR_RGB2GRAY)
        cropped_images.append((class_id, detected_region))

        ocr_result = reader.readtext(detected_region, detail=0)

        selected_texts = []
        for text in ocr_result:
            closest_word = text

            # หาคำใกล้เคียงจากไฟล์ th.txt โดยใช้ระยะทาง Levenshtein
            if class_id == 1:  # เช็คเฉพาะ Province เท่านั้น
                min_distance = float('inf')
                best_match = text

                for th_word in thai_words:
                    dist = lev.distance(text, th_word)
                    if dist < min_distance:
                        min_distance = dist
                        best_match = th_word

                selected_texts.append(best_match)
            else:
                selected_texts.append(text)

        if class_id == 0:
            detected_texts['text_number'].extend(selected_texts)
        elif class_id == 1:
            detected_texts['text_province'].extend(selected_texts)

# แสดงภาพผลลัพธ์พร้อมกล่อง bounding box
plt.figure(figsize=(10, 10))
plt.imshow(output_image)
plt.axis('off')
plt.show()

# แสดงภาพที่ตัดออกมาแต่ละประเภท
for idx, (class_id, cropped_image) in enumerate(cropped_images):
    plt.figure(figsize=(5, 5))
    plt.title(f'Class ID: {class_id}')
    plt.imshow(cropped_image, cmap='gray')
    plt.axis('off')
    plt.show()


# แสดงผลลัพธ์ที่ตรวจจับได้และคำที่ถูกแทนที่จากไฟล์ th.txt เฉพาะ Province
print("Detected Texts - Number:")
for text in detected_texts['text_number']:
    print(text)

print("\nDetected Texts - Province:")
for text in detected_texts['text_province']:
    print(text)

##วีดีโอ

In [ ]:
import sys
import cv2
import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt
import Levenshtein as lev
from Levenshtein import distance
from google.colab.patches import cv2_imshow

sys.path.append('/content/EasyOCR_license_plate')
from easyocr import Reader

# Load the YOLOv8 Model with Custom Weights
model = YOLO('/content/drive/MyDrive/license plate/best.pt')

# Initialize EasyOCR Reader for Thai Language
reader = Reader(['th'], gpu=True)

# Path to the video file
input_video_path = '/content/20240611141829593_fd0afacfcf48424b988fe21221e1e674_AB8230019_0.mp4'  # เปลี่ยนเป็นพาธของไฟล์วีดีโอของคุณ
output_video_path = '/content/output_video.mp4'

# Open the video file
video = cv2.VideoCapture(input_video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (int(video.get(3)), int(video.get(4))))

# Load the Thai words from the dictionary
th_dict_path = '/content/EasyOCR_license_plate/easyocr/dict/th.txt'
with open(th_dict_path, 'r', encoding='utf-8') as f:
    thai_words = [word.strip() for word in f.readlines()]

# Process the video frame by frame
while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform Inference
    results = model.predict(image_rgb, save=True)

    output_image = image_rgb.copy()
    detected_texts = {'text_number': [], 'text_province': []}
    cropped_images = []

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            class_id = int(box.cls)

            cv2.rectangle(output_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            detected_region = cv2.cvtColor(image_rgb[y1:y2, x1:x2], cv2.COLOR_RGB2GRAY)
            cropped_images.append((class_id, detected_region))

            ocr_result = reader.readtext(detected_region, detail=0)

            selected_texts = []
            for text in ocr_result:
                closest_word = text

                if class_id == 1:
                    min_distance = float('inf')
                    best_match = text

                    for th_word in thai_words:
                        dist = lev.distance(text, th_word)
                        if dist < min_distance:
                            min_distance = dist
                            best_match = th_word

                    selected_texts.append(best_match)
                else:
                    selected_texts.append(text)

            if class_id == 0:
                detected_texts['text_number'].extend(selected_texts)
            elif class_id == 1:
                detected_texts['text_province'].extend(selected_texts)

    # Convert the image back to BGR for saving
    output_image_bgr = cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR)
    out.write(output_image_bgr)

    # (Optional) Display the frame
    cv2_imshow(output_image_bgr)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything if job is finished
video.release()
out.release()
cv2.destroyAllWindows()

# แสดงผลลัพธ์ที่ตรวจจับได้และคำที่ถูกแทนที่จากไฟล์ th.txt เฉพาะ Province
print("Detected Texts - Number:")
for text in detected_texts['text_number']:
    print(text)

print("\nDetected Texts - Province:")
for text in detected_texts['text_province']:
    print(text)